<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Text_classification/JobsMessageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libs

In [1]:
!pip install nltk
!pip install emoji --upgrade
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.7 MB/s eta 0:00:00


In [48]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import re
import emoji
import string

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from pymystem3 import Mystem

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Freeze seeds

In [4]:
np.random.seed(42)

## Get the dataset

In [137]:
text_data = pd.read_excel('msg_type.xlsx')

## Tokenization

In [208]:
tokenizer = TweetTokenizer()

## Stemming

In [209]:
stemmer = SnowballStemmer("russian")

## Lemmaization

In [210]:
mystem = Mystem()

## Vectorize using TFIDF

In [211]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian'))

## Split dataset to parameters and encode target labels

In [227]:
df = text_data.copy()
df['category'] = df['category'].replace({'ads': 'message', 'project': 'vacancy'})


In [228]:
df['category']

0      message
1      message
2      message
3      message
4      message
        ...   
475    vacancy
476    vacancy
477    vacancy
478    vacancy
479    vacancy
Name: category, Length: 480, dtype: object

###  Label encode categories

In [229]:
label_encoder = LabelEncoder()
encoded_target = label_encoder.fit_transform(df['category'])

### Split into params and target values

In [230]:
X = df['text']
y = encoded_target

## Perform transformation on df

In [231]:
def remove_emoji(text: str) -> str:
    return emoji.replace_emoji(text, " ")


def remove_links(text: str) -> str:
    return re.sub(r"http\S+", " ", text, flags=re.MULTILINE)


def remove_usernames_and_emails(text: str) -> str:
    """Удалеяет юзернеймы и email"""
    return re.sub(r"\S*@\S*", " ", text, flags=re.MULTILINE)


def remove_punctuation(text: str) -> str:
    """Удаляем символы пунктуации"""
    return "".join([ch if ch not in string.punctuation else " " for ch in text])


def remove_numbers(text: str) -> str:
    """Удаляем числа"""
    return "".join([i if not i.isdigit() else " " for i in text])


def remove_multiple_spaces(text: str) -> str:
    """Удаляем двойные (и более) пробелы"""
    return re.sub(r"\s+", " ", text, flags=re.I)

In [232]:
def get_prep_text(text: str) -> str:
  return remove_multiple_spaces(
      remove_numbers(
          remove_punctuation(
              remove_usernames_and_emails(
                  remove_links(
                      remove_emoji(text)
                      )
                  )
              )
          )
      )

In [238]:
#our new dataset with stemmed lemmatized and later vectorized texts
stemmed_lemma_txts = []

for text in X:
  tok = tokenizer.tokenize(get_prep_text(text).lower())
  stem_tok = [stemmer.stem(token) for token in tok]
  # lem_tok = [lem for lem in mystem.lemmatize(" ".join(stem_tok)) if not lem.isspace()]
  # stemmed_lemma_txts.append(' '.join(lem_tok))
  stemmed_lemma_txts.append(' '.join(stem_tok))

df['text_lemm'] = stemmed_lemma_txts

### TFIDF Vectorize

In [239]:
tfidfd = tfidf_vectorizer.fit_transform(stemmed_lemma_txts)

## Split dataset

pass tfidf'd and transfromed data instead of texts as X

In [240]:
X_train, X_test, y_train, y_test = train_test_split(tfidfd, y, stratify=y, test_size=0.25, random_state=42)

# Model

## Create and train baseline model

In [241]:
model = LogisticRegression(C=0.004)
model.fit(X_train, y_train)

LogisticRegression(C=0.004)

## Predict

In [242]:
y_pred = model.predict(X_test)

## Evaluate

### Accuracy

In [243]:
acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)

Accuracy: 0.9333333333333333


### Report

In [244]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93        60
           1       0.95      0.92      0.93        60

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.93       120
weighted avg       0.93      0.93      0.93       120



## Sum it up

### Catboost

In [245]:
cat_model = CatBoostClassifier(iterations=300, depth=6, learning_rate=0.1, loss_function='MultiClass', verbose=False)
cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))


In [246]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print(f"Accuracy: {accuracy:.2f}")
class_names = ['0','1']
report = classification_report(y_test, y_pred, target_names=class_names)
print("Classification Report:")
print(report)

Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93        60
           1       0.95      0.92      0.93        60

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.93       120
weighted avg       0.93      0.93      0.93       120



In [247]:
def test_model(model):
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  print('Accuracy:', acc)
  print("Classification Report:")
  print(classification_report(y_test, y_pred))


In [248]:
log_model = LogisticRegression(C=0.004)
nb_model = MultinomialNB()
svm_model = SVC(kernel='linear', random_state=42, gamma="auto", probability=True)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = xgb.XGBClassifier()
knn_model = KNeighborsClassifier(n_neighbors=5)

ensemble = VotingClassifier(estimators=[
                              ('rf', rf_model),
                              ('svm', svm_model),
                              ('xgb', xgb_model)],
                            voting='soft')        # soft for probability-based voting

rfensemble = VotingClassifier(estimators=[
                              ('rf', rf_model),
                              ('svm', svm_model),
                              ('rf1', RandomForestClassifier(n_estimators=100, random_state=42))],
                            voting='hard')

models = [log_model, nb_model, svm_model, rf_model, xgb_model, knn_model, ensemble,rfensemble]
model_names = ['logreg', 'bayes', 'SVM', 'RandomForest', 'XGB', 'KNN', 'Ensemble', 'RF_ensemble']

In [249]:
for i in range(len(models)):
  print(model_names[i])
  print()
  test_model(models[i])
  print('\n\n')

logreg

Accuracy: 0.9333333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.95      0.93        60
           1       0.95      0.92      0.93        60

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.93       120
weighted avg       0.93      0.93      0.93       120




bayes

Accuracy: 0.8916666666666667
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.82      0.88        60
           1       0.84      0.97      0.90        60

    accuracy                           0.89       120
   macro avg       0.90      0.89      0.89       120
weighted avg       0.90      0.89      0.89       120




SVM

Accuracy: 0.9416666666666667
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94        60
           1       0.95      0.93      0.94        60



In [250]:
label_encoder.inverse_transform([0, 1])

array(['message', 'vacancy'], dtype=object)

#### Add Stacking of models

In [268]:
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
    ('svm', SVC(kernel='linear', random_state=42, gamma="auto", probability=True)),
     ('logreg', LogisticRegression(C=0.0045)),
    ('xgb', xgb.XGBClassifier())]
final_model = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=42)

stacking_model = StackingClassifier(estimators=base_models, final_estimator=final_model)
test_model(stacking_model)

Accuracy: 0.9583333333333334
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.97      0.96        60
           1       0.97      0.95      0.96        60

    accuracy                           0.96       120
   macro avg       0.96      0.96      0.96       120
weighted avg       0.96      0.96      0.96       120



## Neural networks solution

#### import NN stuff

In [252]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import pad_sequences

### LSTM

#### Prerocessing

Split data

In [253]:
df['text_lemm'][0]

'ищ эксперт кто хочет продава сво услуг быстр и легк привет я ол графическ и веб дизайнер эксперт я созда лаконичн с правильн структур и собствен вайб сайт для эксперт на taplink вся нужн и важн информац собра в одн мест чтоб ваш клиент проход минимальн пут к покупк чтоб узна подробн пиш в лс сотрудничеств'

In [254]:
def k_process(df):
  X = df['text_lemm'].copy()
  y = df['category'].copy()
  return train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

Tokenize

In [255]:
def k_tokenize(x1, x2):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x1)

  x_train = tokenizer.texts_to_sequences(x1)
  x_test = tokenizer.texts_to_sequences(x2)

  return tokenizer, x_train, x_test

Add padding

In [256]:
def k_padme(x1, x2, maxlen=100):
  x_train = pad_sequences(x1, maxlen=maxlen, padding='post')
  x_test = pad_sequences(x2, maxlen=maxlen, padding='post')
  return x_train, x_test

#### Create model

In [257]:
class LSTMClassifier:
  def __init__(self, vocab_size, embedding_dim, maxlen):
    self.vocab = vocab_size
    self.embedding_dim = embedding_dim
    self.maxlen = maxlen
    self.model = self._create_model()

  def _create_model(self)->Sequential:
    '''creates lstm with embedding, rnn and classificator'''
    model = Sequential()
    model.add(Embedding(input_dim=self.vocab, output_dim=self.embedding_dim, input_length=self.maxlen))
    model.add(LSTM(units=64))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


  def fit(self, X_train:np.array, y_train:np.array, batch_size:int=64, epochs:int=5)->None:
    '''trains model'''
    self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)


  def predict(self, X_test:np.array)->np.array:
    '''predics values for validation and test'''
    return self.model.predict(X_test)


  def evaluate(self, pred:np.array, y_test:np.array)->None:
    '''reports statistics'''
    loss, accuracy = self.model.evaluate(pred, y_test)
    pred = (pred > 0.5).astype(int)
    print(f"Loss: {loss}\nAccuracy: {accuracy}")
    print(classification_report(y_test, pred))


  def fit_predict(self, X_train, X_test, y_train, y_test, batch=64, epochs=5):
    '''fits data into model, predicts values and reports sttistics'''
    self.fit(X_train, y_train, batch, epochs)
    y_pred = self.predcit(X_test)
    self.evaluate(y_pred, y_test)

#### Run it

In [258]:
kX_train, kX_test, ky_train, ky_test = k_process(df)
k_tokenizer, kX_train, kX_test = k_tokenize(kX_train, kX_test)
kX_train, kXtest = k_padme(kX_train, kX_test)

In [259]:
k_vocab_size = len(k_tokenizer.word_index) + 1    # num of unique words in all texts +1  for padding token
k_embedding_dim = 100                             #vectors representing a word
maxlen = 100                                      # sequence length after tokenization. too long - cut it

In [260]:
kLSTM = LSTMClassifier(k_vocab_size, k_embedding_dim, maxlen)
kLSTM.fit_predict(kX_train, kX_test, ky_train, ky_test)

Epoch 1/5


UnimplementedError: ignored